### test for using the _plot_unstructured_map_and_save_ function

In [1]:
import os, sys
import shutil
import warnings
warnings.filterwarnings('ignore')

import numpy as np
import xarray as xr
import xesmf as xe

# Helpful for plotting only
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import cartopy
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import uxarray as ux  #need npl 2024a or later
import geoviews.feature as gf

#sys.path.append('/glade/u/home/wwieder/python/adf/lib/plotting_functions.py')
from plotting_functions import *

In [2]:
# Load datataset  
# TODO, develop function for this too
gppfile='/glade/derecho/scratch/wwieder/ADF/b.e30_beta04.BLT1850.ne30_t232_wgx3.121/climo/b.e30_beta04.BLT1850.ne30_t232_wgx3.121_GPP_climo.nc'
mesh0 = '/glade/campaign/cesm/cesmdata/inputdata/share/meshes/ne30pg3_ESMFmesh_cdf5_c20211018.nc'
ds0 = ux.open_dataset(mesh0, gppfile)

In [3]:
# Set up data arrays to plot
# TODO, this should be wrapped into appropriate plotting scripts
spd = 3600*24 # to get bigger fluxes
a = ds0.GPP.isel(time=0) * spd
a.attrs = ds0.GPP.attrs
a.attrs['units'] = 'gC/m2/d'
b = ds0.GPP.isel(time=6) * spd
b.attrs = a.attrs
c = a-b
c.attrs = a.attrs
d = 100*c/b
d.attrs = a.attrs
d.attrs['units'] = '%'

In [4]:
from pathlib import Path
wks = Path("/glade/derecho/scratch/wwieder/testFig")
str(wks)

'/glade/derecho/scratch/wwieder/testFig'

In [8]:
case_nickname = 'jan'
base_nickname = 'july'
case_climo_yrs, baseline_climo_yrs = [10,14],[10,14]
mdlfld = a
obsfld = b
diffld = c
pctld = d
area = ds0.area.isel(time=0)
landfrac = ds0.landfrac.isel(time=0)
wgt = area * landfrac / (area * landfrac).sum()

plot_unstructured_map_and_save(wks, case_nickname, base_nickname,
                               case_climo_yrs, baseline_climo_yrs,
                               mdlfld, obsfld, diffld, pctld, wgt,
                               projection = 'global')
print(str(wks) + '.png made')

<xarray.UxDataArray 'GPP' (n_face: 48600)> Size: 389kB
array([       nan,        nan,        nan, ..., 1.27199839, 1.42239865,
       2.10174724])
Coordinates:
    time     int64 8B 1
Dimensions without coordinates: n_face
Attributes:
    long_name:      gross primary production
    units:          gC/m2/d
    cell_methods:   time: mean
    landunit_mask:  unknown
<xarray.UxDataArray (n_face: 48600)> Size: 194kB
array([          nan,           nan,           nan, ..., 4.2150470e-05,
       1.9083785e-05, 4.9214168e-06], dtype=float32)
Coordinates:
    time     int64 8B 1
Dimensions without coordinates: n_face
/glade/derecho/scratch/wwieder/testFig.png made


In [9]:
# Subset data for polar plots
# set the bounding box
lon_bounds = (-180, 180)
lat_bounds = (50, 90)
# elements include nodes, edge centers, or face centers) 
element = 'face centers'

wks = Path("/glade/derecho/scratch/wwieder/testPolarFig")
case_climo_yrs, baseline_climo_yrs = [10,14],[10,14]
mdlfld = a.subset.bounding_box(lon_bounds, lat_bounds, element=element)
obsfld = b.subset.bounding_box(lon_bounds, lat_bounds, element=element)
diffld = c.subset.bounding_box(lon_bounds, lat_bounds, element=element)
pctld = d.subset.bounding_box(lon_bounds, lat_bounds, element=element)
area = ds0.area.subset.bounding_box(lon_bounds, lat_bounds, element=element).isel(time=0)
landfrac = ds0.landfrac.subset.bounding_box(lon_bounds, lat_bounds, element=element).isel(time=0)
wgt = area * landfrac / (area * landfrac).sum()

plot_unstructured_map_and_save(wks, case_nickname, base_nickname,
                               case_climo_yrs, baseline_climo_yrs,
                               mdlfld, obsfld, diffld, pctld, wgt,
                               projection = 'polar')
print(str(wks) + '.png made')

/glade/derecho/scratch/wwieder/testPolarFig.png made
